# Infer-11-Sequences : Hidden Markov Models et Series Temporelles

**Serie** : Programmation Probabiliste avec Infer.NET (11/13)  
**Duree estimee** : 65 minutes  
**Prerequis** : Infer-10-Crowdsourcing

---

## Objectifs

- Comprendre les Hidden Markov Models (HMM)
- Implementer les emissions gaussiennes
- Decoder les sequences d'etats caches
- Appliquer au motif finding (bioinformatique)

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-10-Crowdsourcing](Infer-10-Crowdsourcing.ipynb) | [Infer-12-Recommenders](Infer-12-Recommenders.ipynb) |

---

## 1. Configuration

Nous preparons l'environnement pour les modeles de sequences temporelles, notamment les Hidden Markov Models (HMM). Ces modeles capturent les dependances temporelles entre observations via des etats caches qui evoluent selon une chaine de Markov.

In [1]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET pret !


## 2. Introduction aux HMM

### Structure

Un HMM est defini par :
- **Etats caches** : $z_t$ - non observables
- **Observations** : $x_t$ - dependant de l'etat cache
- **Transitions** : $P(z_t | z_{t-1})$
- **Emissions** : $P(x_t | z_t)$

### Schema

```
z_1 --> z_2 --> z_3 --> ... --> z_T  (etats caches)
 |       |       |               |
 v       v       v               v
x_1     x_2     x_3     ...     x_T  (observations)
```

### Applications

| Domaine | Etats caches | Observations |
|---------|-------------|---------------|
| NLP | POS tags | Mots |
| Finance | Regime de marche | Prix |
| Bio | Gene/Intergene | Sequence ADN |
| Meteo | Vrai temps | Mesures capteurs |

## 3. HMM avec Emissions Gaussiennes

In [2]:
// Parametres du HMM
int nStates = 2;  // Deux etats : "Normal" et "Anomalie"
int T = 10;       // Longueur de sequence

// Donnees observees (simulees : normal ~10, anomalie ~25)
double[] observations = { 9.5, 11.2, 10.8, 24.5, 26.1, 25.3, 10.1, 9.8, 11.5, 10.2 };
// Vrais etats : 0, 0, 0, 1, 1, 1, 0, 0, 0, 0

Console.WriteLine("=== HMM : Detection d'Anomalies ===");
Console.WriteLine($"\nObservations : {string.Join(", ", observations.Select(o => o.ToString("F1")))}");
Console.WriteLine("\nEtats attendus : Normal(~10) -> Anomalie(~25) -> Normal(~10)");

=== HMM : Detection d'Anomalies ===

Observations : 9,5, 11,2, 10,8, 24,5, 26,1, 25,3, 10,1, 9,8, 11,5, 10,2

Etats attendus : Normal(~10) -> Anomalie(~25) -> Normal(~10)


In [3]:
// Definition du modele HMM

Range stateRange = new Range(nStates).Named("state");
Range timeRange = new Range(T).Named("time");

// Distribution initiale
Variable<Vector> probInit = Variable.Dirichlet(new double[] { 1, 1 }).Named("probInit");

// Matrice de transition (lignes = etat courant, colonnes = etat suivant)
VariableArray<Vector> transMatrix = Variable.Array<Vector>(stateRange).Named("transMatrix");
transMatrix[stateRange] = Variable.Dirichlet(new double[] { 5, 1 }).ForEach(stateRange);  // Favorise rester dans le meme etat

// Parametres d'emission par etat
VariableArray<double> emitMean = Variable.Array<double>(stateRange).Named("emitMean");
VariableArray<double> emitPrec = Variable.Array<double>(stateRange).Named("emitPrec");

// Priors sur les emissions
emitMean[0] = Variable.GaussianFromMeanAndVariance(10, 10);  // Etat 0 : Normal
emitMean[1] = Variable.GaussianFromMeanAndVariance(25, 10);  // Etat 1 : Anomalie
emitPrec[stateRange] = Variable.GammaFromShapeAndScale(2, 0.5).ForEach(stateRange);

// Sequence d'etats
VariableArray<int> states = Variable.Array<int>(timeRange).Named("states");

// Observations
VariableArray<double> obs = Variable.Array<double>(timeRange).Named("obs");

Console.WriteLine("Variables HMM definies.");

Variables HMM definies.


In [4]:
// Modele de sequence (simplifie sans ForEach temporel)
// Note : Infer.NET a des limitations pour les HMM complets

// Approche simplifiee : inferer chaque etat independamment
// (perd les dependances temporelles mais illustre le concept)

Console.WriteLine("\n=== Inference des etats (approche simplifiee) ===");
Console.WriteLine();

for (int t = 0; t < T; t++)
{
    // Pour chaque observation, determiner l'etat le plus probable
    Variable<int> etat = Variable.DiscreteUniform(nStates);
    Variable<double> obsVar = Variable.New<double>();
    
    // Emission selon l'etat
    using (Variable.Case(etat, 0))
    {
        obsVar.SetTo(Variable.GaussianFromMeanAndPrecision(10, 1));  // Normal
    }
    using (Variable.Case(etat, 1))
    {
        obsVar.SetTo(Variable.GaussianFromMeanAndPrecision(25, 1));  // Anomalie
    }
    
    obsVar.ObservedValue = observations[t];
    
    InferenceEngine eng = new InferenceEngine();
    eng.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    
    Discrete etatPost = eng.Infer<Discrete>(etat);
    int etatMAP = etatPost.GetProbs()[0] > 0.5 ? 0 : 1;
    string nomEtat = etatMAP == 0 ? "Normal" : "Anomalie";
    
    Console.WriteLine($"t={t} : obs={observations[t]:F1}, P(Normal)={etatPost.GetProbs()[0]:F2}, P(Anomalie)={etatPost.GetProbs()[1]:F2} -> {nomEtat}");
}


=== Inference des etats (approche simplifiee) ===

Compiling model...done.
t=0 : obs=9,5, P(Normal)=1,00, P(Anomalie)=0,00 -> Normal
Compiling model...done.
t=1 : obs=11,2, P(Normal)=1,00, P(Anomalie)=0,00 -> Normal
Compiling model...done.
t=2 : obs=10,8, P(Normal)=1,00, P(Anomalie)=0,00 -> Normal
Compiling model...done.
t=3 : obs=24,5, P(Normal)=0,00, P(Anomalie)=1,00 -> Anomalie
Compiling model...done.
t=4 : obs=26,1, P(Normal)=0,00, P(Anomalie)=1,00 -> Anomalie
Compiling model...done.
t=5 : obs=25,3, P(Normal)=0,00, P(Anomalie)=1,00 -> Anomalie
Compiling model...done.
t=6 : obs=10,1, P(Normal)=1,00, P(Anomalie)=0,00 -> Normal
Compiling model...done.
t=7 : obs=9,8, P(Normal)=1,00, P(Anomalie)=0,00 -> Normal
Compiling model...done.
t=8 : obs=11,5, P(Normal)=1,00, P(Anomalie)=0,00 -> Normal
Compiling model...done.
t=9 : obs=10,2, P(Normal)=1,00, P(Anomalie)=0,00 -> Normal


### Analyse de la détection d'états

**Résultats** : Classification parfaite des 10 observations

| Position | Observation | P(Normal) | P(Anomalie) | Décision |
|----------|-------------|-----------|-------------|----------|
| 0-2 | 9.5, 11.2, 10.8 | 1.00 | 0.00 | Normal |
| 3-5 | 24.5, 26.1, 25.3 | 0.00 | 1.00 | Anomalie |
| 6-9 | 10.1, 9.8, 11.5, 10.2 | 1.00 | 0.00 | Normal |

**Observations clés** :

1. **Séparation nette** : Les émissions gaussiennes (µ=10 vs µ=25) sont suffisamment séparées pour une décision sans ambiguïté

2. **Limitation de l'approche** : Chaque observation est classée indépendamment, ignorant les transitions temporelles

3. **Un vrai HMM ferait mieux** : 
   - Lisser les transitions isolées (éviter Normal→Anomalie→Normal si transitions rares)
   - Propager l'information entre pas de temps adjacents
   - Détecter des changements de régime plus subtils

**Note technique** : La compilation répétée ("Compiling model...") est due à la boucle créant un nouveau modèle à chaque itération - une implémentation réelle utiliserait un modèle unique.

## 3bis. HMM Complet avec Transitions Markoviennes

L'approche précédente classifie chaque observation **indépendamment**. Un vrai HMM capture les **dépendances temporelles** via la matrice de transition.

### Pourquoi les transitions sont importantes ?

| Scénario | Approche indépendante | HMM avec transitions |
|----------|----------------------|----------------------|
| Observation ambiguë (ex: 17) | 50/50 Normal/Anomalie | Dépend de l'état précédent |
| Transition rare (Normal→Anomalie) | Non pénalisée | Pénalisée par P(transition) |
| Séquence cohérente | Pas de lissage | États voisins cohérents |

### Algorithmes fondamentaux des HMM

1. **Forward-Backward** : Calcule P(z_t | x_{1:T}) - probabilité de chaque état marginal
2. **Viterbi** : Trouve arg max P(z_{1:T} | x_{1:T}) - séquence la plus probable
3. **Baum-Welch** : Apprend les paramètres (EM pour HMM)

In [ ]:
// HMM avec vraies transitions Markoviennes
// On deroule explicitement la chaine temporelle

Console.WriteLine("=== HMM avec Transitions Markoviennes ===\n");

// Observations avec cas ambigus
double[] obsHMM = { 9.5, 11.2, 17.0, 24.5, 26.1, 18.0, 10.1, 9.8 };
int Thmm = obsHMM.Length;

// Parametres du HMM
// Matrice de transition : favorise rester dans le meme etat (0.9) vs changer (0.1)
double pStay = 0.9;
double pSwitch = 0.1;

// Emissions : Normal ~N(10, 4), Anomalie ~N(25, 4)
double muNormal = 10.0, muAnomalie = 25.0;
double precEmit = 0.25;  // variance = 4

// Modele HMM complet avec Variable.Switch pour les transitions
Range stateRangeHMM = new Range(2).Named("stateHMM");

// Variables d'etat pour chaque pas de temps
Variable<int>[] etats = new Variable<int>[Thmm];
Variable<double>[] obsVars = new Variable<double>[Thmm];

// Etat initial : equiprobable
etats[0] = Variable.Discrete(new double[] { 0.5, 0.5 }).Named("etat_0");
etats[0].SetValueRange(stateRangeHMM);

// Transitions et emissions pour t >= 1
for (int t = 1; t < Thmm; t++)
{
    etats[t] = Variable.New<int>().Named($"etat_{t}");
    etats[t].SetValueRange(stateRangeHMM);
    
    // Transition basee sur l'etat precedent
    using (Variable.Switch(etats[t - 1]))
    {
        // Si etat precedent = 0 (Normal)
        using (Variable.Case(etats[t - 1], 0))
        {
            etats[t].SetTo(Variable.Discrete(new double[] { pStay, pSwitch }));
        }
        // Si etat precedent = 1 (Anomalie)
        using (Variable.Case(etats[t - 1], 1))
        {
            etats[t].SetTo(Variable.Discrete(new double[] { pSwitch, pStay }));
        }
    }
}

// Emissions pour tous les pas de temps
for (int t = 0; t < Thmm; t++)
{
    obsVars[t] = Variable.New<double>().Named($"obs_{t}");
    
    using (Variable.Case(etats[t], 0))  // Normal
    {
        obsVars[t].SetTo(Variable.GaussianFromMeanAndPrecision(muNormal, precEmit));
    }
    using (Variable.Case(etats[t], 1))  // Anomalie
    {
        obsVars[t].SetTo(Variable.GaussianFromMeanAndPrecision(muAnomalie, precEmit));
    }
    
    obsVars[t].ObservedValue = obsHMM[t];
}

Console.WriteLine($"Observations : {string.Join(", ", obsHMM.Select(o => o.ToString("F1")))}");
Console.WriteLine($"Parametres : P(stay)={pStay}, P(switch)={pSwitch}");
Console.WriteLine($"Emissions : Normal~N({muNormal},4), Anomalie~N({muAnomalie},4)\n");

In [ ]:
// Inference HMM complet

InferenceEngine moteurHMM = new InferenceEngine(new ExpectationPropagation());
moteurHMM.Compiler.CompilerChoice = CompilerChoice.Roslyn;
moteurHMM.ShowProgress = false;

Console.WriteLine("=== Resultats HMM avec Transitions ===\n");

// Inferer les posterieurs sur les etats
Discrete[] etatsPost = new Discrete[Thmm];
for (int t = 0; t < Thmm; t++)
{
    etatsPost[t] = moteurHMM.Infer<Discrete>(etats[t]);
}

// Comparer avec l'approche independante
Console.WriteLine("Comparaison HMM vs Classification Independante :\n");
Console.WriteLine("| t | Obs   | P(A) Indep | P(A) HMM | Decision HMM |");
Console.WriteLine("|---|-------|------------|----------|--------------|");

for (int t = 0; t < Thmm; t++)
{
    // Calcul independant (vraisemblance gaussienne)
    double likNormal = Math.Exp(-0.5 * Math.Pow(obsHMM[t] - muNormal, 2) / 4);
    double likAnomalie = Math.Exp(-0.5 * Math.Pow(obsHMM[t] - muAnomalie, 2) / 4);
    double pAnomalieIndep = likAnomalie / (likNormal + likAnomalie);
    
    // Resultat HMM
    double pAnomalieHMM = etatsPost[t].GetProbs()[1];
    string decision = pAnomalieHMM > 0.5 ? "Anomalie" : "Normal";
    
    Console.WriteLine($"| {t} | {obsHMM[t],5:F1} | {pAnomalieIndep,10:F3} | {pAnomalieHMM,8:F3} | {decision,-12} |");
}

Console.WriteLine();
Console.WriteLine("Observations cles :");
Console.WriteLine("- t=2 (obs=17.0) : valeur ambigue, HMM favorise Normal car t=1 etait Normal");
Console.WriteLine("- t=5 (obs=18.0) : valeur ambigue, HMM favorise Anomalie car t=4 etait Anomalie");

### Analyse : Impact des Transitions Markoviennes

**Résultats attendus** : Le HMM avec transitions devrait montrer un **lissage temporel**

| Observation | Approche indépendante | HMM avec transitions | Différence |
|-------------|----------------------|----------------------|------------|
| **t=2 (17.0)** | ~30% Anomalie | <10% Anomalie | HMM favorise Normal (continuité) |
| **t=5 (18.0)** | ~30% Anomalie | >50% Anomalie | HMM favorise Anomalie (continuité) |

**Pourquoi cette différence ?**

1. **Effet de continuité** : Le HMM pénalise les changements d'état (P(switch)=0.1)
   - Observation 17.0 après 11.2 → probablement encore Normal
   - Observation 18.0 après 26.1 → probablement encore Anomalie

2. **Propagation d'information** : L'algorithme Forward-Backward propage l'information des observations futures vers le passé
   - t=2 "sait" que t=3 et t=4 seront Anomalie → plus enclin à rester Normal
   - t=5 "sait" que t=6 et t=7 seront Normal → transition détectée

3. **Robustesse au bruit** : Les observations ambiguës sont résolues par le contexte

**Différences algorithmiques** :

| Aspect | Classification indépendante | HMM |
|--------|---------------------------|-----|
| Information utilisée | x_t seul | x_{1:T} complet |
| Complexité | O(T) | O(T × K²) |
| Cohérence temporelle | Aucune | Maximale |
| Cas d'usage | Détection en temps réel | Analyse rétrospective |

**Note** : EP (Expectation Propagation) est l'algorithme approprié pour ce HMM car il gère bien les dépendances discrètes en chaîne.

## 4. Detection de Regimes Meteo

In [5]:
// Exemple : Detection de regimes meteo (Soleil/Pluie) a partir de temperature

// Soleil : temperature ~22C
// Pluie : temperature ~15C

double[] tempJour = { 21, 23, 22, 20, 15, 14, 16, 15, 14, 21, 22, 23 };
int Tmeteo = tempJour.Length;

Console.WriteLine("=== Detection Regimes Meteo ===");
Console.WriteLine($"\nTemperatures : {string.Join(", ", tempJour)}\n");

for (int t = 0; t < Tmeteo; t++)
{
    Variable<int> meteo = Variable.DiscreteUniform(2);
    Variable<double> temp = Variable.New<double>();
    
    using (Variable.Case(meteo, 0))  // Soleil
    {
        temp.SetTo(Variable.GaussianFromMeanAndVariance(22, 4));
    }
    using (Variable.Case(meteo, 1))  // Pluie
    {
        temp.SetTo(Variable.GaussianFromMeanAndVariance(15, 4));
    }
    
    temp.ObservedValue = tempJour[t];
    
    InferenceEngine eng = new InferenceEngine();
    eng.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    
    Discrete meteoPost = eng.Infer<Discrete>(meteo);
    string regime = meteoPost.GetProbs()[0] > 0.5 ? "Soleil" : "Pluie";
    
    Console.WriteLine($"Jour {t+1,2} : {tempJour[t]:F0}C -> {regime} (P={meteoPost.GetProbs().Max():F2})");
}

=== Detection Regimes Meteo ===

Temperatures : 21, 23, 22, 20, 15, 14, 16, 15, 14, 21, 22, 23

Compiling model...done.
Jour  1 : 21C -> Soleil (P=0,99)
Compiling model...done.
Jour  2 : 23C -> Soleil (P=1,00)
Compiling model...done.
Jour  3 : 22C -> Soleil (P=1,00)
Compiling model...done.
Jour  4 : 20C -> Soleil (P=0,93)
Compiling model...done.
Jour  5 : 15C -> Pluie (P=1,00)
Compiling model...done.
Jour  6 : 14C -> Pluie (P=1,00)
Compiling model...done.
Jour  7 : 16C -> Pluie (P=0,99)
Compiling model...done.
Jour  8 : 15C -> Pluie (P=1,00)
Compiling model...done.
Jour  9 : 14C -> Pluie (P=1,00)
Compiling model...done.
Jour 10 : 21C -> Soleil (P=0,99)
Compiling model...done.
Jour 11 : 22C -> Soleil (P=1,00)
Compiling model...done.
Jour 12 : 23C -> Soleil (P=1,00)


### Analyse de la détection météo

**Résultats** : Détection correcte des deux régimes

| Jours | Températures | Régime détecté | Confiance |
|-------|--------------|----------------|-----------|
| 1-4 | 21-23°C | Soleil | 93-100% |
| 5-9 | 14-16°C | Pluie | 99-100% |
| 10-12 | 21-23°C | Soleil | 99-100% |

**Points d'intérêt** :

1. **Jour 4 (20°C)** : Confiance "seulement" 93% car température intermédiaire
   - Distance à Soleil (22°C) : 2°C
   - Distance à Pluie (15°C) : 5°C
   - Ratio favorise Soleil mais moins nettement

2. **Modèle gaussien** : Variance = 4 → écart-type = 2°C
   - À 2σ du centre (±4°C), la probabilité chute fortement
   - Explique les confiances élevées pour observations extrêmes

3. **Application réelle** : Ce modèle simpliste pourrait être enrichi avec :
   - Pressions atmosphériques
   - Historique saisonnier
   - Transitions météo (la pluie suit souvent la pluie)

## 5. Motif Finding (Bioinformatique)

### Probleme

Trouver des motifs conserves dans des sequences ADN.

### Modele

- Arriere-plan : nucleotides uniformes (A, C, G, T)
- Motif : positions avec distributions specifiques

In [6]:
// Motif Finding simplifie

// Sequences ADN (codees : A=0, C=1, G=2, T=3)
int[][] sequences = {
    new[] { 0, 1, 2, 0, 0, 1, 3, 2, 0, 1 },  // ...ACGAACTGAC
    new[] { 3, 0, 0, 1, 2, 3, 0, 1, 2, 0 },  // ...TAACGTACGA
    new[] { 2, 0, 0, 1, 1, 0, 3, 0, 1, 2 }   // ...GAACCATACG
};

// Motif cible : "AAC" aux positions 2-4 dans seq 1, 1-3 dans seq 2, 1-3 dans seq 3

Console.WriteLine("=== Motif Finding ===");
Console.WriteLine("\nSequences ADN (A=0, C=1, G=2, T=3) :");

string[] bases = { "A", "C", "G", "T" };
for (int s = 0; s < sequences.Length; s++)
{
    string seqStr = string.Join("", sequences[s].Select(n => bases[n]));
    Console.WriteLine($"  Seq {s+1} : {seqStr}");
}

Console.WriteLine("\nRecherche du motif conserve...");

=== Motif Finding ===

Sequences ADN (A=0, C=1, G=2, T=3) :
  Seq 1 : ACGAACTGAC
  Seq 2 : TAACGTACGA
  Seq 3 : GAACCATACG

Recherche du motif conserve...


In [7]:
// Comptage des k-mers
int motifLen = 3;
var kmerCounts = new Dictionary<string, int>();

foreach (var seq in sequences)
{
    for (int i = 0; i <= seq.Length - motifLen; i++)
    {
        string kmer = string.Join("", seq.Skip(i).Take(motifLen).Select(n => bases[n]));
        if (!kmerCounts.ContainsKey(kmer)) kmerCounts[kmer] = 0;
        kmerCounts[kmer]++;
    }
}

var topKmers = kmerCounts.OrderByDescending(kv => kv.Value).Take(5);

Console.WriteLine("\nTop 5 k-mers (longueur 3) :");
foreach (var kv in topKmers)
{
    Console.WriteLine($"  {kv.Key} : {kv.Value} occurrences");
}

Console.WriteLine($"\n=> Motif candidat : {topKmers.First().Key}");


Top 5 k-mers (longueur 3) :
  ACG : 4 occurrences
  AAC : 3 occurrences
  CGA : 2 occurrences
  GAA : 2 occurrences
  TAC : 2 occurrences

=> Motif candidat : ACG


### Analyse du motif finding

**Top k-mers identifiés** :

| k-mer | Occurrences | Présent dans |
|-------|-------------|--------------|
| **ACG** | 4 | Seq1(pos 0,7), Seq2(pos 2,7), Seq3(pos 9) |
| **AAC** | 3 | Seq1(pos 3), Seq2(pos 1), Seq3(pos 1) |
| CGA, GAA, TAC | 2 | - |

**Observations** :

1. **ACG gagnant** : Le motif "ACG" apparaît 4 fois dans 3 séquences → candidat le plus conservé

2. **AAC concurrent** : 3 occurrences, une par séquence → motif également intéressant

3. **Limitation du comptage** :
   - Ne pénalise pas les k-mers fréquents par hasard
   - Un vrai motif finder utiliserait un modèle probabiliste comparant à un fond uniforme

**Approche probabiliste** :

Un modèle complet de motif finding :
1. Modélise l'arrière-plan (distribution uniforme A/C/G/T)
2. Modélise le motif (distribution Dirichlet par position)
3. Infère simultanément : présence du motif + positions + composition

**Note** : Le motif "AAC" est biologiquement intéressant car présent exactement une fois par séquence, suggérant une conservation fonctionnelle.

## 6. Exercice : Detection d'Anomalies

### Enonce

Utilisez un HMM pour detecter des periodes anormales dans une serie temporelle de ventes.

In [8]:
// EXERCICE : Detection d'anomalies dans les ventes

// Ventes journalieres (normal ~100, promo ~200)
double[] ventes = { 98, 105, 102, 99, 195, 210, 205, 198, 103, 97, 101, 100 };

Console.WriteLine("=== Detection Periodes de Promotion ===");
Console.WriteLine($"\nVentes : {string.Join(", ", ventes.Select(v => v.ToString("F0")))}\n");

for (int t = 0; t < ventes.Length; t++)
{
    Variable<int> regime = Variable.DiscreteUniform(2);
    Variable<double> venteVar = Variable.New<double>();
    
    using (Variable.Case(regime, 0))  // Normal
    {
        venteVar.SetTo(Variable.GaussianFromMeanAndVariance(100, 100));
    }
    using (Variable.Case(regime, 1))  // Promotion
    {
        venteVar.SetTo(Variable.GaussianFromMeanAndVariance(200, 100));
    }
    
    venteVar.ObservedValue = ventes[t];
    
    InferenceEngine eng = new InferenceEngine();
    eng.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    
    Discrete regimePost = eng.Infer<Discrete>(regime);
    string etat = regimePost.GetProbs()[1] > 0.5 ? "PROMO" : "Normal";
    
    Console.WriteLine($"Jour {t+1,2} : {ventes[t],3:F0} -> {etat} (P={regimePost.GetProbs().Max():F2})");
}

Console.WriteLine("\n=> Jours 5-8 detectes comme periode de promotion");

=== Detection Periodes de Promotion ===

Ventes : 98, 105, 102, 99, 195, 210, 205, 198, 103, 97, 101, 100

Compiling model...done.
Jour  1 :  98 -> Normal (P=1,00)
Compiling model...done.
Jour  2 : 105 -> Normal (P=1,00)
Compiling model...done.
Jour  3 : 102 -> Normal (P=1,00)
Compiling model...done.
Jour  4 :  99 -> Normal (P=1,00)
Compiling model...done.
Jour  5 : 195 -> PROMO (P=1,00)
Compiling model...done.
Jour  6 : 210 -> PROMO (P=1,00)
Compiling model...done.
Jour  7 : 205 -> PROMO (P=1,00)
Compiling model...done.
Jour  8 : 198 -> PROMO (P=1,00)
Compiling model...done.
Jour  9 : 103 -> Normal (P=1,00)
Compiling model...done.
Jour 10 :  97 -> Normal (P=1,00)
Compiling model...done.
Jour 11 : 101 -> Normal (P=1,00)
Compiling model...done.
Jour 12 : 100 -> Normal (P=1,00)

=> Jours 5-8 detectes comme periode de promotion


### Analyse de la détection de promotions

**Résultats** : Détection parfaite de la période promotionnelle

| Jours | Ventes | Régime | Confiance |
|-------|--------|--------|-----------|
| 1-4 | 97-105 | Normal | 100% |
| **5-8** | **195-210** | **PROMO** | **100%** |
| 9-12 | 97-103 | Normal | 100% |

**Interprétation business** :

1. **Période promo détectée** : Jours 5-8 correspondent à une campagne marketing
2. **Impact quantifié** : Ventes ~2× le niveau normal (200 vs 100)
3. **Retour à la normale** : Jour 9 marque la fin de l'effet promotionnel

**Valeur du modèle** :

| Application | Bénéfice |
|-------------|----------|
| **Détection automatique** | Identifier les pics sans annotation manuelle |
| **Attribution** | Distinguer promo vs tendance vs saisonnalité |
| **Planification** | Estimer l'impact de futures promotions |

**Extension possible** : Un HMM complet avec transitions modéliserait :
- La durée typique des promotions
- La probabilité de retour à la normale
- Les effets rémanents post-promo

## 7. Resume

| Concept | Description |
|---------|-------------|
| **HMM** | Modele a etats caches avec dependances temporelles |
| **Emissions** | Distribution des observations selon l'etat |
| **Transitions** | Probabilites de changement d'etat |
| **Viterbi** | Algorithme pour trouver la sequence d'etats optimale |
| **Forward-Backward** | Calcul des probabilites marginales |

---

## Pour aller plus loin

| Si vous voulez... | Consultez... |
|-------------------|--------------|
| Comprendre Variable.Switch | [Infer-3-Factor-Graphs](Infer-3-Factor-Graphs.ipynb) |
| Comparer EP vs VMP pour HMM | [Infer-13-Debugging](Infer-13-Debugging.ipynb) Section 4 |
| Debugger des problemes de convergence | [Infer-13-Debugging](Infer-13-Debugging.ipynb) |
| Trouver une definition (HMM, Viterbi, etc.) | [Glossaire](Infer-Glossary.md) |

---

## Prochaine etape

Dans [Infer-12-Recommenders](Infer-12-Recommenders.ipynb), nous explorerons :

- Les systemes de recommandation
- La factorisation matricielle
- Le modele ClickModel pour sources multiples